# Library Imports


In [1]:
# Local imports
from finetuned_prompt import finetuned_prompt
from langchain import hub
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage

# from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from pinotdb.sqlalchemy import PinotDialect, PinotHTTPDialect, PinotHTTPSDialect
from rich.markdown import Markdown
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from sqlalchemy.dialects import registry
from dotenv import load_dotenv
import os

# Initialize Agent and Prompt


In [2]:
load_dotenv(".env")
registry.register("pinot", "pinotdb.sqlalchemy", "PinotDialect")
PinotDialect.supports_statement_cache = False
PinotHTTPSDialect.supports_statement_cache = False

db = SQLDatabase.from_uri(os.environ["PINOT_SQLALCHEMY_URI"])

llm = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o-mini", temperature=0
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
# print(toolkit.get_tools())

# Pull prompt
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

system_message = (
    prompt_template.format(dialect="Apache Pinot MYSQL_ANSI dialect", top_k=3)
    + finetuned_prompt
)
# print(system_message)


# Initialize a Rich Console for pretty console outputs
console = Console()

# print prompt
console.print(
    Panel(
        system_message,
        width=console.width,
        style="bold cyan",
    ),
    style="bold cyan",
)

# Create agent
agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

c:\Users\rishi\Desktop\git\funnel_analysis_agent\.venv\Lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ System: You are an agent designed to interact with a SQL database.                                              │
│ Given an input question, create a syntactically correct Apache Pinot MYSQL_ANSI dialect query to run, then look │
│ at the results of the query and return the answer.                                                              │
│ Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most │
│ 3 results.                                                                                                      │
│ You can order the results by a relevant column to return the most interesting examples in the database.         │
│ Never query for all the columns from a specific table, only ask for the relevant columns given the question.    │
│ You have access to tools for interacting with the database.                                                     │
│ Only use the below tools. Only use the information returned by the below tools to construct your final answer.  │
│ You MUST double check your query before executing it. If you get an error while executing a query, rewrite the  │
│ query and try again.                                                                                            │
│                                                                                                                 │
│ DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.                             │
│                                                                                                                 │
│ To start you should ALWAYS look at the tables in the database to see what you can query.                        │
│ Do NOT skip this step.                                                                                          │
│ Then you should query the schema of the most relevant tables.                                                   │
│                                                                                                                 │
│ NOTE: Here is some extra info that you need to keep in mind:                                                    │
│ 1. The queries you generate should be in accordance with the Calcite SQL parser and use the MySQL ANSI Dialect, │
│ as they are going to run on Apache Pinot.                                                                       │
│ 2. Always try to use the tools available to you:                                                                │
│     - `sql_db_list_tables` -  to get all the table names.                                                       │
│     - `sql_db_schema` - to get the column schema for all tables in the list of tables.                          │
│     - `sql_db_query_checker` -  to double check your generated query before executing it.                       │
│     - `sql_db_query` - to finally execute the query and return the result.                                      │
│ 3. Description of tables:                                                                                       │
│     a. "clickstream_events" - this contains transactional info about user events (view, click, save, purchase), │
│ when they were performed and the duration. The timestamp is stored as milliseconds from epoch.                  │
│     b. "purchase_info" - this contains transactional info about purchases performed by users (buyers).          │
│     c. "NewProducts" - this is the Products dimensional table - contains info about the items and their         │
│ descriptions.                                                                                                   │
│     d. "Users" - this is the Users dimensional table.                                                           │
│ 4. Your SQL query responses are required to answer que

# Ask Questions to the Agent


In [3]:
def print_results(events):
    # stream output
    for event in events:
        last_message = event["messages"][-1]
        last_message: BaseMessage

        if last_message.type == "human":
            last_message: HumanMessage
            continue

        elif last_message.type == "ai":
            last_message: AIMessage
            if (
                last_message.tool_calls
                and last_message.tool_calls[-1]["name"] == "sql_db_query"
            ):
                query = last_message.tool_calls[-1]["args"]["query"]
                to_print = Markdown(
                    f"""Generated Query:

```sql
{query}
```""",
                    inline_code_lexer="sql",
                )
                # Display the generated SQL query
                console.print(to_print, markup=True, highlight=True)
                continue

        elif last_message.type == "tool":
            last_message: ToolMessage
            if last_message.name in [
                "sql_db_schema",
                "sql_db_list_tables",
                "sql_db_query_checker",
            ]:
                continue
            elif last_message.name == "sql_db_query":
                to_print = Text(
                    f"Query Result:\n{last_message.content}", style="bold green"
                )
                # Display the SQL Query Result
                console.print(to_print)
                continue
        else:
            print("Unknown Message Type, just printing out as it is.")

        to_print = (
            Markdown(f"Agent: {last_message.content}")
            if last_message.content != ""
            else Text(last_message.pretty_repr(), style="bold green")
        )

        # Display the agent's response
        console.print(to_print, markup=True, highlight=True)

In [4]:
def invoke_model(user_input: str):
    print("You:", user_input)

    # invoke model
    events = agent_executor.stream(
        {"messages": [("user", user_input)]},
        stream_mode="values",
    )

    print_results(events)

## Query the Agent


1. Question 1: What is the overall funnel conversion rate?


In [5]:
user_input = "What is the overall funnel conversion rate?"
invoke_model(user_input)

You: What is the overall funnel conversion rate?


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_FlhxUqAGBdOLebSzxNUS5fdc)
 Call ID: call_FlhxUqAGBdOLebSzxNUS5fdc
  Args:

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_yAZ53MmaVuZ1s198E0sWj7dP)
 Call ID: call_yAZ53MmaVuZ1s198E0sWj7dP
  Args:
    table_names: clickstream_events

================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_Tnt3DPSs95hJBeHUKohqqNe1)
 Call ID: call_Tnt3DPSs95hJBeHUKohqqNe1
  Args:
    query: SELECT 
    FUNNEL_COUNT(
        STEPS(
            event_type = 'view', 
            event_type = 'click', 
            event_type = 'save', 
            event_type = 'purchase'
        ),
        CORRELATE_BY(user_id)
    ) AS funnel_counts
FROM clickstream_events;

Generated Query:                                                                                                   

                                                                                                                   
 SELECT                                                                                                            
     FUNNEL_COUNT(                                                                                                 
         STEPS(                                                                                                    
             event_type = 'view',                                                                                  
             event_type = 'click',                                                                                 
             event_type = 'save',                                                                                  
             event_type = 'purchase'                                                                               
         ),                                                                                                        
         CORRELATE_BY(user_id)                                                                                     
     ) AS funnel_counts                                                                                            
 FROM clickstream_events;                                                                                          
                                                                                                                   

Query Result:
[([100, 100, 100, 100],)]

Agent: The overall funnel conversion rate is 100%. This means that all users who entered the funnel by viewing a   
product completed all the steps, including clicking, saving, and purchasing the product.

2. Question 2: What is the biggest drop-off in the funnel?


In [6]:
user_input = "What is the biggest drop-off in the funnel?"
invoke_model(user_input)

You: What is the biggest drop-off in the funnel?


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_GQ1csPGP1yNInVLOAIoV8EIa)
 Call ID: call_GQ1csPGP1yNInVLOAIoV8EIa
  Args:

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_EUR0oyw3RAJElUc34UcslyeD)
 Call ID: call_EUR0oyw3RAJElUc34UcslyeD
  Args:
    table_names: clickstream_events

================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_Eo52h0IcGcAX53pwfNEXXeHG)
 Call ID: call_Eo52h0IcGcAX53pwfNEXXeHG
  Args:
    query: SELECT FUNNEL_COUNT(STEPS(event_type = 'view', event_type = 'click', event_type = 'save', event_type = 
'purchase'), CORRELATE_BY(user_id)) AS counts FROM clickstream_events

Generated Query:                                                                                                   

                                                                                                                   
 SELECT FUNNEL_COUNT(STEPS(event_type = 'view', event_type = 'click', event_type = 'save', event_type = 'purchase' 
 CORRELATE_BY(user_id)) AS counts FROM clickstream_events                                                          
                                                                                                                   

Query Result:
[([100, 100, 100, 100],)]

Agent: The funnel analysis shows that there is no drop-off between the steps in the funnel. All users who viewed a 
product also clicked, saved, and purchased it. Therefore, the biggest drop-off in the funnel is zero.

3. Question 3: Who are the top 3 users in terms of time spent?


In [7]:
user_input = "Who are the top 3 users in terms of time spent?"
invoke_model(user_input)

You: Who are the top 3 users in terms of time spent?


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kqyjEyDqiQ6IKJQBpQmD5wY2)
 Call ID: call_kqyjEyDqiQ6IKJQBpQmD5wY2
  Args:

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_uzU8r8hlBoyLaRAtYu0JLqqn)
 Call ID: call_uzU8r8hlBoyLaRAtYu0JLqqn
  Args:
    table_names: clickstream_events, Users

================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_1Qi60CXAIVSGvXTBol8Yg8HU)
 Call ID: call_1Qi60CXAIVSGvXTBol8Yg8HU
  Args:
    query: SELECT user_id, SUM(duration) as total_time_spent FROM clickstream_events GROUP BY user_id ORDER BY 
total_time_spent DESC LIMIT 3

Generated Query:                                                                                                   

                                                                                                                   
 SELECT user_id, SUM(duration) as total_time_spent FROM clickstream_events GROUP BY user_id ORDER BY               
 total_time_spent DESC LIMIT 3                                                                                     
                                                                                                                   

...

Query Result:
Error: (pinotdb.exceptions.DatabaseError) {'errorCode': 700,
 'message': 'QueryValidationError:\nData source (FROM clause) not found'}
[SQL: SELECT lookup('ws_2opqcdizwoh9.Users', 'Name', 'ID', 'b51d0d34-cfff-4e17-a3d8-6794f3ecdda6') as user_name]
(Background on this error at: https://sqlalche.me/e/20/4xp6)

Agent: I encountered an issue while trying to retrieve the names of the top users using the lookup function. It    
seems that the function requires a FROM clause to work correctly. Let me try a different approach to get the names 
of these users.

Generated Query:                                                                                                   

                                                                                                                   
 SELECT Name FROM Users WHERE ID IN ('498ac520-1df2-47de-a287-47724fb5718b', 'e4aa4fac-0e3d-4402-b476-bbd92bcc12bd 
 'b51d0d34-cfff-4e17-a3d8-6794f3ecdda6')                                                                           
                                                                                                                   

Query Result:
[('Michael Diaz',), ('Lauren Johnson',), ('Nathan Morris',)]

Agent: The top 3 users in terms of time spent are:                                                                 

 1 Michael Diaz                                                                                                    
 2 Lauren Johnson                                                                                                  
 3 Nathan Morris

4. Question 4: What other products can we recommend to these top users? For context, also show the names and descriptions of the top products of the users, and also of the recommended products.


In [8]:
user_input = "What other products can we recommend to these top users? For context, also show the names and descriptions of the top products of the users, and also of the recommended products."
invoke_model(user_input)

You: What other products can we recommend to these top users? For context, also show the names and descriptions of the top products of the users, and also of the recommended products.


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_a3PZjEw5NxnHaBKp2Z2dmgZg)
 Call ID: call_a3PZjEw5NxnHaBKp2Z2dmgZg
  Args:

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_ZZx0hvOWe5UnRfY3Xj9IEykH)
 Call ID: call_ZZx0hvOWe5UnRfY3Xj9IEykH
  Args:
    table_names: clickstream_events, NewProducts, Users

================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_SincY5KRaLuophavpIq5cQlc)
 Call ID: call_SincY5KRaLuophavpIq5cQlc
  Args:
    query: SELECT user_id, SUM(duration) as total_duration FROM clickstream_events GROUP BY user_id ORDER BY 
total_duration DESC LIMIT 3;

Generated Query:                                                                                                   

                                                                                                                   
 SELECT user_id, SUM(duration) as total_duration FROM clickstream_events GROUP BY user_id ORDER BY total_duration  
 DESC LIMIT 3;                                                                                                     
                                                                                                                   

Query Result:
[('498ac520-1df2-47de-a287-47724fb5718b', 51900554.0), ('e4aa4fac-0e3d-4402-b476-bbd92bcc12bd', 51148857.0), 
('b51d0d34-cfff-4e17-a3d8-6794f3ecdda6', 51071024.0)]

Generated Query:                                                                                                   

                                                                                                                   
 SELECT product_id, SUM(duration) as total_duration FROM clickstream_events WHERE user_id =                        
 'b51d0d34-cfff-4e17-a3d8-6794f3ecdda6' GROUP BY product_id ORDER BY total_duration DESC LIMIT 1;                  
                                                                                                                   

Query Result:
[('96e45fc1-46fd-4ced-9530-f5205c9b48b5', 128757.0)]

Generated Query:                                                                                                   

                                                                                                                   
 SELECT Name, Description, embedding FROM NewProducts WHERE ID = '96e45fc1-46fd-4ced-9530-f5205c9b48b5';           
                                                                                                                   

Agent: Here are the top products for the top users and the recommended similar products:                           

 1 Top User 1:                                                                                                     
    • Top Product: Rather Mixer                                                                                    
       • Description: A high-quality mixer perfect for home & kitchen enthusiasts.                                 
    • Recommended Products:                                                                                        
       • Right Mixer: A high-quality mixer perfect for home & kitchen enthusiasts.                                 
       • Area Mixer: A high-quality mixer perfect for home & kitchen enthusiasts.                                  
       • Dog Mixer: A high-quality mixer perfect for home & kitchen enthusiasts.                                   
 2 Top User 2:                                                                                                     
    • Top Product: Analysis Jeans                                                                                  
       • Description: A high-quality jeans perfect for clothing enthusiasts.                                       
    • Recommended Products:                                                                                        
       • Watch Jeans: A high-quality jeans perfect for clothing enthusiasts.                                       
       • Probably Jeans: A high-quality jeans perfect for clothing enthusiasts.                                    
       • Behavior Jeans: A high-quality jeans perfect for clothing enthusiasts.                                    

Unfortunately, there was an error while fetching recommendations for the third top user. If you need further       
assistance or a retry, please let me know!

5. Question 5: What are the top 5 electronic items sold?


In [9]:
user_input = "What are the top 5 electronic items sold?"
invoke_model(user_input)

You: What are the top 5 electronic items sold?


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Ird50t4i7O1hHWqgRnQdwqir)
 Call ID: call_Ird50t4i7O1hHWqgRnQdwqir
  Args:

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_AxkOjvuSiGwgVCgf7FiASO99)
 Call ID: call_AxkOjvuSiGwgVCgf7FiASO99
  Args:
    table_names: purchase_info, NewProducts

================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_bwroChrniPyEOXja461IvF59)
 Call ID: call_bwroChrniPyEOXja461IvF59
  Args:
    query: SELECT product_id, SUM(quantity) as total_quantity FROM purchase_info GROUP BY product_id ORDER BY 
total_quantity DESC LIMIT 5

Generated Query:                                                                                                   

                                                                                                                   
 SELECT product_id, SUM(quantity) as total_quantity FROM purchase_info GROUP BY product_id ORDER BY total_quantity 
 DESC LIMIT 5                                                                                                      
                                                                                                                   

...

Generated Query:                                                                                                   

                                                                                                                   
 SELECT lookup('ws_2opqcdizwoh9.NewProducts', 'Name', 'ID', 'cbd7b61c-eb68-4ea2-839a-cc3cf973e1e9') as product_nam 
                                                                                                                   

Query Result:
Error: (pinotdb.exceptions.DatabaseError) {'errorCode': 700,
 'message': 'QueryValidationError:\nData source (FROM clause) not found'}
[SQL: SELECT lookup('ws_2opqcdizwoh9.NewProducts', 'Name', 'ID', 'cbd7b61c-eb68-4ea2-839a-cc3cf973e1e9') as 
product_name]
(Background on this error at: https://sqlalche.me/e/20/4xp6)

Generated Query:                                                                                                   

                                                                                                                   
 SELECT Name FROM NewProducts WHERE ID IN ('caf51933-9d2a-478d-b031-3f4f1f90f943',                                 
 'ed0a39d4-073f-4001-9d63-824aea64adaf', 'de674661-93cd-46da-856a-68787e6bcfd4',                                   
 '69633395-e8ae-4f96-b358-e949b49489b6', 'cbd7b61c-eb68-4ea2-839a-cc3cf973e1e9')                                   
                                                                                                                   

Query Result:
[('Media Lego Set',), ('College Action Figure',), ('News Puzzle',), ('Fact Puzzle',), ('Themselves Puzzle',)]

Agent: The top 5 electronic items sold are:                                                                        

 1 Media Lego Set                                                                                                  
 2 College Action Figure                                                                                           
 3 News Puzzle                                                                                                     
 4 Fact Puzzle                                                                                                     
 5 Themselves Puzzle